# Inference

## Dependencies and imports

In [2]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

from openai import OpenAI

## Utility

In [4]:
PROMPT = "Sei un dipendente pubblico che deve scrivere dei documenti istituzionali italiani per renderli semplici e comprensibili per i cittadini. Ti verrà fornito un documento pubblico e il tuo compito sarà quello di riscriverlo applicando regole di semplificazione senza però modificare il significato del documento originale. Ad esempio potresti rendere le frasi più brevi, eliminare le perifrasi, esplicitare sempre il soggetto, utilizzare parole più semplicii, trasformare i verbi passivi in verbi di forma attiva, spostare le frasi parentetiche alla fine del periodo."

In [6]:
openai_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def build_prompt(_text_to_simplify):
    messages = [
        {"role": "system", "content": PROMPT},
        {"role": "user", "content": _text_to_simplify},
    ]
    return messages


def predict(_texts_to_simplify):
    prompts = [build_prompt(_text) for _text in _texts_to_simplify]

    outputs = []
    for prompt in tqdm(prompts):
        response = openai_client.chat.completions.create(
            model='gpt-4-turbo-2024-04-09',
            messages=prompt,
            stream=False,
            temperature=0.2,
            top_p=0.1
        )
        outputs.append(response.choices[0].message.content)
    return outputs

## Load datasets

In [7]:
df = pd.read_csv('./texts/original.csv',encoding='utf-8')

## Random predictions

In [9]:
for s in df.sample(1).to_dict(orient='records'):
  output = predict([s['original_text']])[0]
  print("\noriginal: ", s['original_text'])
  print("\nmodel: ", output)
  print('----------------')

100%|██████████| 1/1 [00:03<00:00,  3.19s/it]


original:  A cadenza annuale, immediatamente dopo l'erogazione del servizio ed in maniera assolutamente anonima e riservata, agli utenti viene consegnato un questionario da compilare, per rilevare la valutazione degli stessi utenti sulla qualità del servizio reso. 

model:  Ogni anno, subito dopo aver ricevuto il servizio, diamo agli utenti un questionario anonimo e confidenziale da compilare. Questo serve per sapere cosa pensano della qualità del servizio che abbiamo offerto.
----------------


## Run all predictions

In [10]:
df['simplified_text'] = predict(df['original_text'].tolist())

100%|██████████| 619/619 [1:02:44<00:00,  6.08s/it]


## Save simplified datasets

In [11]:
df.to_csv('./texts/gpt4.csv', index=False)